In [1]:
import urllib.request
import requests
import zipfile
import io
import os
import glob
import pandas as pd
import boto
import boto.s3
import sys
import luigi
import datetime, time
import numpy as np
import seaborn as sns

from boto.s3.key import Key
from urllib.request import urlopen
from bs4 import BeautifulSoup as bsoup

In [2]:
def is_file_present(directory,filename):
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_list = glob.glob(directory+'//*.csv')
    for file_name_in_dir in file_list:
        if (directory+ '\\' + filename) == (file_name_in_dir+".zip"):
            return True
    return False

In [3]:
def download_data(data_type):
    base_URL = "https://resources.lendingclub.com"
    url = urllib.request.urlopen("https://www.lendingclub.com/info/download-data.action")
    content = url.read()
    soup= bsoup(content,'lxml')
    
    #find div by ID
    fileNameDiv = soup.find('div',{"id":data_type})
    FileList = fileNameDiv.text.rstrip("|")

    for fileName in FileList.split("|"):
        file_URL= base_URL+'/'+fileName
        print(file_URL)
        if not is_file_present(data_type,fileName):    
            zfile = requests.get(file_URL)
            z = zipfile.ZipFile(io.BytesIO(zfile.content))
            z.extractall(data_type)

In [4]:
def read_data(directory):
    fileList = glob.glob(directory+'//*.csv')
    
    dfList=[]
    #columns=["id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_il_6m","open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il","il_util","open_rv_12m","open_rv_24m","max_bal_bc","all_util","total_rev_hi_lim","inq_fi","total_cu_tl",
             #"inq_last_12m","acc_open_past_24mths","avg_cur_bal","bc_open_to_buy","bc_util","chargeoff_within_12_mths","delinq_amnt","mo_sin_old_il_acct","mo_sin_old_rev_tl_op","mo_sin_rcnt_rev_tl_op","mo_sin_rcnt_tl","mort_acc","mths_since_recent_bc","mths_since_recent_bc_dlq","mths_since_recent_inq","mths_since_recent_revol_delinq","num_accts_ever_120_pd","num_actv_bc_tl","num_actv_rev_tl","num_bc_sats","num_bc_tl","num_il_tl","num_op_rev_tl","num_rev_accts","num_rev_tl_bal_gt_0","num_sats","num_tl_120dpd_2m","num_tl_30dpd","num_tl_90g_dpd_24m","num_tl_op_past_12m","pct_tl_nvr_dlq","percent_bc_gt_75","pub_rec_bankruptcies","tax_liens","tot_hi_cred_lim","total_bal_ex_mort","total_bc_limit","total_il_high_credit_limit"]
    for filename in fileList:
        print(filename)
        df=pd.read_csv(filename, low_memory=False,skiprows=1)
        print(df.shape)
        ts = time.time()
        df["download_timestamp"] = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
        df["recorded_timestamp"] = filename.rstrip('csv').lstrip('loanStatsFileNamesJS\\').lstrip('rejectedLoanStatsFileNamesJS\\').lstrip('loanStatsFileNamesJS/').lstrip('rejectedLoanStatsFileNamesJS/').lstrip("LoanStats").lstrip("RejectStats").lstrip("_")
        dfList.append(df)
    concatDf=pd.concat(dfList, axis=0)
    #concatDf.columns=columns
    concatDf.to_csv(directory+"_concat_file.csv", index=None)
    print(concatDf.shape)
    return concatDf

In [5]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    return mis_val_table_ren_columns

In [6]:
def clean_rejected_data(data):
    x100 = data[pd.notnull(data['Risk_Score'])]
    x10=x100.groupby("State")["Risk_Score"].mean().reset_index(name="Risk_Score_mean")
    x10['Risk_Score_mean'] = pd.to_numeric(x10['Risk_Score_mean'])

    data["Employment Length Value"]=data["Employment Length"].str.extract("(\d+)",expand=True)
    data['Employment Length Value'] = pd.to_numeric(data['Employment Length Value'])

    data["DTI"]=data["Debt-To-Income Ratio"].str.rstrip("%")
    data['DTI'] = pd.to_numeric(data['DTI'])

    x11=data.groupby("State")["Employment Length Value"].mean().reset_index(name="Employment Length Value Mean")
    x12=pd.merge(data,x10,how='left')
    data=pd.merge(x12,x11,how='left')
    
    data["Risk_Score"]=(data["Risk_Score_mean"]+43).where((data["Risk_Score"].isnull()) & (data["DTI"]>43),data["Risk_Score_mean"])
    
    data["Risk_Score"]=(data["Risk_Score"]+93).where((data["Risk_Score"].isnull()) & (data["DTI"]>43) &(data["Employment Length Value"]<data["Employment Length Value Mean"]),data["Risk_Score_mean"])
    
    data["Risk_Score"]=(data["Risk_Score_mean"]).where((data["Risk_Score"].isnull()),data["Risk_Score_mean"])
    
    data['Loan Title'] = data['Loan Title'].fillna('Others')
    
    data['State'] = data[['State']].convert_objects(convert_numeric=True).fillna('Other')
    
    data['Zip Code'] = data[['Zip Code']].convert_objects(convert_numeric=True).fillna('000xx')
    
    #Converting to datatime format
    data["Application Date"]=pd.to_datetime(data['Application Date'])
    data['Application Date Month'] = data['Application Date'].dt.month
    data['Application Date Year'] = data['Application Date'].dt.year
    return data

In [7]:
rejected_loan = 'rejectedLoanStatsFileNamesJS'
download_data(rejected_loan)
rejectedData = read_data(rejected_loan)
rejectedData.to_csv("Rejected_Loans_Combined_Data.csv")

https://resources.lendingclub.com/RejectStatsA.csv.zip
https://resources.lendingclub.com/RejectStatsB.csv.zip
https://resources.lendingclub.com/RejectStatsD.csv.zip
https://resources.lendingclub.com/RejectStats_2016Q1.csv.zip
https://resources.lendingclub.com/RejectStats_2016Q2.csv.zip
https://resources.lendingclub.com/RejectStats_2016Q3.csv.zip
https://resources.lendingclub.com/RejectStats_2016Q4.csv.zip
rejectedLoanStatsFileNamesJS\RejectStatsA.csv
(755491, 9)
rejectedLoanStatsFileNamesJS\RejectStatsB.csv
(2694642, 9)
rejectedLoanStatsFileNamesJS\RejectStatsD.csv
(2859379, 9)
rejectedLoanStatsFileNamesJS\RejectStats_2016Q1.csv
(1096204, 9)
rejectedLoanStatsFileNamesJS\RejectStats_2016Q2.csv
(996561, 9)
rejectedLoanStatsFileNamesJS\RejectStats_2016Q3.csv
(1272619, 9)
rejectedLoanStatsFileNamesJS\RejectStats_2016Q4.csv
(1404490, 9)
(11079386, 11)


In [8]:
loanData = pd.read_csv("Rejected_Loans_Combined_Data.csv", encoding= 'iso-8859-1',low_memory=False)
#print amount of missing values
print('Missing values in loan rejected data')
print(missing_values_table(loanData))
loanData = clean_rejected_data(loanData)
#Write clean data to new CSV
loanData.to_csv("Cleaned_Rejected_Loans_Combined_Data.csv")

Missing values in loan rejected data
                      Missing Values  % of Total Values
Unnamed: 0                         0           0.000000
Amount Requested                   0           0.000000
Application Date                   0           0.000000
Loan Title                       197           0.001778
Risk_Score                   6402779          57.790017
Debt-To-Income Ratio               0           0.000000
Zip Code                         291           0.002626
State                             22           0.000199
Employment Length                  0           0.000000
Policy Code                        0           0.000000
download_timestamp                 0           0.000000
recorded_timestamp                 0           0.000000


C:\Users\prith\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\prith\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [ ]:
AWS_ACCESS_KEY_ID = input("Please enter AWS access key")
AWS_SECRET_ACCESS_KEY = input("Please enter AWS secret key")

bucket_name = 'lending-club-analysis'

conn = boto.connect_s3(AWS_ACCESS_KEY_ID,
        AWS_SECRET_ACCESS_KEY)

bucket = conn.create_bucket(bucket_name,
    location=boto.s3.connection.Location.DEFAULT)

   
file = "Cleaned_Rejected_Loans_Combined_Data.csv"
print ('Uploading %s to Amazon S3 bucket %s' % \
   (file, bucket_name))

def percent_cb(complete, total):
    sys.stdout.write('.')
    sys.stdout.flush()

k = Key(bucket)
k.key = file
k.set_contents_from_filename(file,
    cb=percent_cb, num_cb=10)